# DLS Assignment - 4

### This assignment was discussed with Tejasram Ramesh and Ramki Ramamurthy.

### For this assignment I referred the following resources:
  #### 1) Deep Learning with Python - by Francois Cholet
  #### 2) https://www.tensorflow.org/api_docs/python/tf/custom_gradient
  #### 3) https://www.tensorflow.org/tutorials/customization/custom_layers
  #### 4) https://www.coursera.org/lecture/custom-models-layers-loss-functions-with-tensorflow/coding-your-own-custom-dense-layer-PZxhA
  #### 5) https://www.tensorflow.org/guide/autodiff

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
# import tensorflow_addons as tfa
# import librosa
# import librosa.display
import timeit
from IPython.display import Audio
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import soundfile as sf
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling1D,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten
from keras.models import Model
import pickle
import tarfile

### Connecting to GPU

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.186999797999988
GPU (s):
0.04028142600000706
GPU speedup over CPU: 79x


### Loading the MNIST Dataset

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
#Lets look at the training data
print("Training Images Shape: ",train_images.shape)
print("Training Labels: ",train_labels)


#Lets look at the testing data
print("Testing Images Shape: ",test_images.shape)
print("Testing Labels: ",test_labels)

Training Images Shape:  (60000, 28, 28)
Training Labels:  [5 0 4 ... 5 6 8]
Testing Images Shape:  (10000, 28, 28)
Testing Labels:  [7 2 1 ... 4 5 6]


### Normalizing the Images

In [ ]:
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images/=255
test_images/=255

### Loading the Model from Part 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_path = '/content/drive/MyDrive/DLS_Assignments/Models/assign4_part1_baseline.h5'

In [ ]:
baseline = keras.models.load_model(model_path)

In [ ]:
evaluation = baseline.evaluate(test_images , test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1257 - sparse_categorical_accuracy: 0.9856


In [ ]:
W1 = baseline.get_layer(index=1).get_weights()[0]
W2 = baseline.get_layer(index=2).get_weights()[0]
W3 = baseline.get_layer(index=3).get_weights()[0]
W4 = baseline.get_layer(index=4).get_weights()[0]
W5 = baseline.get_layer(index=5).get_weights()[0]


B1 = baseline.get_layer(index=1).get_weights()[1]
B2 = baseline.get_layer(index=2).get_weights()[1]
B3 = baseline.get_layer(index=3).get_weights()[1]
B4 = baseline.get_layer(index=4).get_weights()[1]
B5 = baseline.get_layer(index=5).get_weights()[1]

### Creating the Model

In [ ]:
@tf.custom_gradient
def customgrad(weight):
  s_1 , U_1, v_1 = tf.linalg.svd(weight)
  S_1 = tf.linalg.diag(s_1)

  V_1 = tf.transpose(v_1)

  W = tf.matmul(tf.matmul(U_1[:,:20], S_1[:20,:20]), V_1[:20,:])

    #backprop
  def grad(dy):
    gradient = dy
    return gradient

  return W, grad




class CustomDense(Layer):
  def __init__(self, units, activation,W,B):
        super(CustomDense, self).__init__()
        self.units = units
        W = tf.Variable(W, name='Weights')
        B = tf.Variable(B, name='Biases')
        self.w = W
        self.b = B
        self.activation=activation


  # def weight_biases(self, input_shape,W,B):
  #       #w_init = tf.random_normal_initializer()


  def call(self, inputs):
        W_svd = customgrad(self.w)
        out = tf.matmul(inputs,W_svd) + self.b
        return self.activation(out)

In [ ]:
model_3 = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    CustomDense(1024,tf.nn.relu,W1,B1),
    CustomDense(1024,tf.nn.relu,W2,B2),
    CustomDense(1024,tf.nn.relu,W3,B3),
    CustomDense(1024,tf.nn.relu,W4,B4),
    CustomDense(1024,tf.nn.relu,W5,B5),
    layers.Dense(10, name='output' ,activation='softmax')
])

print(model_3.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 custom_dense_35 (CustomDens  (None, 1024)             803840    
 e)                                                              
                                                                 
 custom_dense_36 (CustomDens  (None, 1024)             1049600   
 e)                                                              
                                                                 
 custom_dense_37 (CustomDens  (None, 1024)             1049600   
 e)                                                              
                                                                 
 custom_dense_38 (CustomDens  (None, 1024)             1049600   
 e)                                                   

### Training the Model

In [ ]:
model_3.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

history_3 = model_3.fit(train_images, train_labels, batch_size=512, epochs=10,verbose=True)

Epoch 1/10
118/118 [==============================] - 342s 3s/step - loss: 0.3432 - accuracy: 0.9054
Epoch 2/10
118/118 [==============================] - 338s 3s/step - loss: 0.0867 - accuracy: 0.9768
Epoch 3/10
118/118 [==============================] - 337s 3s/step - loss: 0.0676 - accuracy: 0.9806
Epoch 4/10
118/118 [==============================] - 337s 3s/step - loss: 0.0537 - accuracy: 0.9844
Epoch 5/10
118/118 [==============================] - 337s 3s/step - loss: 0.0538 - accuracy: 0.9839
Epoch 6/10
118/118 [==============================] - 337s 3s/step - loss: 0.0527 - accuracy: 0.9843
Epoch 7/10
118/118 [==============================] - 337s 3s/step - loss: 0.0512 - accuracy: 0.9847
Epoch 8/10
118/118 [==============================] - 337s 3s/step - loss: 0.0475 - accuracy: 0.9850
Epoch 9/10
118/118 [==============================] - 337s 3s/step - loss: 0.0467 - accuracy: 0.9854
Epoch 10/10
118/118 [==============================] - 336s 3s/step - loss: 0.0524 - accura

In [ ]:
evaluation = model_3.evaluate(test_images , test_labels)

313/313 [==============================] - 894s 3s/step - loss: 0.1040 - accuracy: 0.9743
